# Single Particle


Simulate the trajectory of an individual point, first in a perfectly circular flow (a.k.a. solid body rotation). Then add a convergent term to obtain a spiraling trajectory, and a constant vertical velocity for the third dimension. These simple flow configurations can be thought of as idealized models e.g. ocean meso-scale eddies.

For additional documentation e.g. see :
[1](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/),
[2](https://JuliaClimate.github.io/MeshArrays.jl/dev/),
[3](https://docs.juliadiffeq.org/latest/solvers/ode_solve.html),
[4](https://en.wikipedia.org/wiki/Displacement_(vector))

![solid body rotation](https://github.com/JuliaClimate/IndividualDisplacements.jl/raw/master/examples/figs/SolidBodyRotation.gif)

# 1 Problem Configuration

Here we set up software, grid, flow fields, initial conditions.

### 1.1 Import Software

In [1]:
using IndividualDisplacements, DataFrames
p=dirname(pathof(IndividualDisplacements))
include(joinpath(p,"../examples/flow_fields.jl"))

Main.##258.test2_periodic_domain

### 1.2  Flow Fields

The `simple_flow_field` function (defined in `helper_functions.jl`) defines a simple
three-dimensional flow field. Exercise: locate `simple_flow_field` and modify the
flow field parameters (e.g. intensity and sign of the convergent term).

In [2]:
np,nz=16,4 #gridded domain size (horizontal and vertical)

u,v,w=solid_body_rotation(np,nz) #staggered velocity arrays

𝐹=𝐹_Array3D{eltype(u)}(u,u,v,v,0*w,1*w,[0,19.95*2*pi]); #FlowFields data structure

### 1.3 Initialize Individuals

Let's just set up one individual at [np*1/3,np*1/3,nz*1/3] in the three-dimensional
space where the flow fields have been configured

In [3]:
(x,y,z)=(np*1/3,np*1/3,nz*1/3)

𝐼=Individuals(𝐹,x,y,z)

  📌 details     = (3,) Float64
  🔴 details     = (0, 5) ["ID", "x", "y", "z", "t"]
  🆔 range       = (1, 1)
  🚄 function    = dxyz_dt
  ∫  function    = default_solver
  🔧 function    = 🔧
  𝑃  details     = (:u0, :u1, :v0, :v1, :w0, :w1, :𝑇)


### 1.4 A Closer Look (optional)

The above `Individuals` constructor wraps up 𝐹, the initial position, and other needed components
within 𝐼. **At this point, you can either jump to section 2 or read through this section**
to learn more about how the details as needed e.g. if you wanted to overide default options
that were selected for you by the section 1.3 constructor.

Initial position is

In [4]:
📌=[x,y,z]

3-element Array{Float64,1}:
 5.333333333333333
 5.333333333333333
 1.3333333333333333

and the data structure ([DataFrame](http://juliadata.github.io/DataFrames.jl/stable/))
to record properties along the individual's path accordingly.

In [5]:
🔴 = DataFrame(ID=Int[], x=Float64[], y=Float64[], z=Float64[], t=Float64[])

,ID,x,y,z,t
,Int64,Float64,Float64,Float64,Float64


It is the postprocessing function's responsibility to provide the record. It is thus
important that this intermediary (`postproc`) be consistent with the solver setup (`sol`)
and the expected record format (`🔴`).

In [6]:
function postproc(sol,𝐹::FlowFields;id=missing,𝑇=missing)
    df=postprocess_xy(sol,𝐹,id=id,𝑇=𝑇)
    #add third coordinate
    z=sol[3,:]
    df.z=z[:]
    return df
end

postproc (generic function with 1 method)

The velocity function `🚄` relies only on flow fields obtained from
`𝐹` (which is defined above) to interpolate velocity at the specified
space-time position (e.g. those of individuals).

In [7]:
🚄 = dxyz_dt

dxyz_dt (generic function with 3 methods)

Now that every thing needed to carry out the computation is in place,
we wrap up the problem configuration in a struct (`Individuals`) which
links to the initial positions, flow fields, etc. all that will be
necessary to compute trajectories over time (`∫!(𝐼,𝑇)`).

In [8]:
#assemble as a NamedTuple:
I=(position=📌,record=🔴,velocity=🚄,
postprocessing=postproc,parameters=𝐹)

#construct Individuals from NamedTuple:
𝐼=Individuals(I)

  📌 details     = (3,) Float64
  🔴 details     = (0, 5) ["ID", "x", "y", "z", "t"]
  🆔 range       = (1, 1)
  🚄 function    = dxyz_dt
  ∫  function    = default_solver
  🔧 function    = postproc
  𝑃  details     = (:u0, :u1, :v0, :v1, :w0, :w1, :𝑇)


## 2 Trajectory Simulations

The `∫!` function call below returns the final positions & updates `𝐼.📌` accordingly. It also records properties observed along the trajectory in `𝐼.🔴`.
Simple methods to visualize the individual trajectory (plot or movie) are provided at the end.

### 2.1 Compute Trajectories

In [9]:
𝑇=(0.0,𝐼.𝑃.𝑇[2])
∫!(𝐼,𝑇)

3-element Array{Float64,1}:
 7.721088772857553
 9.468917662354212
 0.7065856056430158

### 2.2 Visualize Trajectories

- define `myplot` convenience function
- generate animation using `myplot`
- single plot example using `myplot`

Single plot example:

Animation example:

Exercise: make the sinking velocity decrease with time
(hint: it increases as specified above in the original notebook);
change the number of times the particle goes around the origin; etc

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*